In [75]:
import pandas as pd
from pandasql import sqldf

# Import data

In [57]:
# file path to data folder
paths = {'windows': 'C:/Users/jjung/iCloudDrive/ETH/MSc 3rd semester/Semester project/Data',
         'macOS': '/Users/janoschjungo/Library/Mobile Documents/com~apple~CloudDocs/ETH/MSc 3rd semester/Semester project/Data'}
path = paths['windows']

In [58]:
# import physiological data
physio = pd.DataFrame()
for subjectID in range(1, 28):
    # load physiological data for subject
    try:
        file = path + f'/subjectID_{subjectID}.csv'
        physio_subject = pd.read_csv(file)
    except FileNotFoundError:
        path = paths['macOS']
        file = path + f'/subjectID_{subjectID}.csv'
        physio_subject = pd.read_csv(file)
    n_entries = physio_subject.shape[0]
    physio_subject['SubjectID'] = [subjectID for i in range(n_entries)]

    # clean column names (dataset contains different column names per subject)
    if 'SkinTemperature.Value' in physio_subject.columns:
        physio_subject = physio_subject.rename(columns={'SkinTemperature.Value': 'SkinTemperature'}, errors='raise')

    # combine all subject data
    physio = pd.concat([physio, physio_subject])

# import fatigue (PROs) data
fatigue = pd.read_csv(path + '/fatiguePROs.csv')

# Convert data

# a) Fatigue (PROs) data

In [59]:
# convert questions into fatigue variables
query = '''
SELECT SubjectID as subjectID, DateTime AS timestamp, Timezone AS timezone,
CASE
    WHEN PROquestion LIKE 'Describe fatigue on a scale of 1 to 10, where 1 means you don’t feel tired at all and 10 means the worst tiredness you can imagine' THEN PROanswer_value
END AS 'VAS',
CASE
    WHEN PROquestion LIKE 'Physically, today how often did you feel exhausted?' THEN PROanswer_choice
END AS 'phF',
CASE
    WHEN PROquestion LIKE 'Mentally, today how often did you feel exhausted?' THEN PROanswer_choice
END AS 'MF',
CASE
    WHEN PROquestion LIKE 'Are you feeling better, worse or the same as yesterday?' THEN PROanswer_choice
END AS 'ReIP',
CASE
    WHEN PROquestion LIKE 'Did you do sport today?' THEN PROanswer_choice
END AS 'sport'
FROM fatigue
'''
temp = sqldf(query)
temp = temp.fillna(value=pd.NA)
print('total number of questions:', temp.shape[0])

total number of questions: 2271


In [60]:
# distinguish different questionnaires with same timestamp (note: data is in chronological order)
temp = temp.reset_index(drop=False)
temp = temp.rename(columns={'index': 'id'})

# same id = same questionnaire
questionnaire = 0
questionnaire_timestamp = temp.loc[0, 'timestamp']
asked_questions = {'VAS': 0, 'phF': 0, 'MF': 0, 'ReIP': 0, 'sport': 0} # already asked?
for i, row in temp.iterrows():
    asked_questions = {key: value + int(not pd.isna(row[key])) for key, value in asked_questions.items()}

    # different timestamp? -> different questionnaire
    if row['timestamp'] != questionnaire_timestamp:
        questionnaire += 1
        questionnaire_timestamp = temp.loc[i, 'timestamp']
        asked_questions = {key: int(not pd.isna(row[key])) for key, value in asked_questions.items()}
    # same timestamp but same question again (vas already asked)? -> different questionnaire
    elif 2 in set(asked_questions.values()):
        questionnaire += 1
        questionnaire_timestamp = temp.loc[i, 'timestamp']
        asked_questions = {key: int(not pd.isna(row[key])) for key, value in asked_questions.items()}
    else:
        pass

    temp.loc[i, 'id'] = questionnaire

query = '''
SELECT id, subjectID, timestamp, GROUP_CONCAT(DISTINCT timezone) AS timezone, GROUP_CONCAT(VAS) AS VAS, GROUP_CONCAT(phF) AS phF, GROUP_CONCAT(MF) AS MF, GROUP_CONCAT(ReIP) AS ReIP, GROUP_CONCAT(sport) AS sport
FROM temp
GROUP BY id, subjectID, timestamp
'''
questionnaires = sqldf(query)
print('total number of separate questionnaires:', questionnaires.shape[0])

total number of separate questionnaires: 526


In [61]:
# incomplete questionnaires (ignore sport label as it's not asked in all questionnaires)
query = '''
SELECT *
FROM questionnaires
WHERE VAS IS NULL OR phF IS NULL OR MF IS NULL OR ReIP IS NULL
'''
temp = sqldf(query)

# discard incomplete questionnaires
query = '''
SELECT *
FROM questionnaires

EXCEPT

SELECT *
FROM questionnaires
WHERE VAS IS NULL OR phF IS NULL OR MF IS NULL OR ReIP IS NULL;
'''
questionnaires = sqldf(query)

print('number of discarded questionnaires:', temp.shape[0])
temp

number of discarded questionnaires: 3


,id,subjectID,timestamp,timezone,VAS,phF,MF,ReIP,sport
0,105,15,09.06.19 22:27,UTC,1.0,None,None,None,None
1,215,23,05.02.18 21:00,CET,None,None,None,Better,None
2,281,24,06.02.18 21:43,CET,1.0,None,None,None,None


In [62]:
# aggregate questionnaires into daily fatigue values
query = '''
SELECT subjectID, SUBSTRING(Timestamp, 1, 8) AS date, GROUP_CONCAT(DISTINCT timezone) AS timezone, GROUP_CONCAT(VAS) AS VAS, GROUP_CONCAT(ReIP) AS ReIP, GROUP_CONCAT(phF) AS phF, GROUP_CONCAT(MF) AS MF, GROUP_CONCAT(sport) AS sport, COUNT(*) AS n_answers
FROM questionnaires
GROUP BY subjectID, date
ORDER BY n_answers DESC;
'''
fatigue_daily = sqldf(query)

# check how many multiple questionnaires are filled out per day
query = '''
SELECT COUNT(*) AS same_day_questionnaires
FROM fatigue_daily
WHERE n_answers > 1;
'''
print('total number of same day questionnaires:', sqldf(query)['same_day_questionnaires'][0])

total number of same day questionnaires: 44


In [63]:
# convert fatigue answers into numeric classes
query = '''
SELECT subjectID, timestamp, timezone,
CASE
WHEN CAST(VAS AS INT) BETWEEN 1.0 AND 4.0 THEN 0
WHEN CAST(VAS AS INT) BETWEEN 5.0 AND 10.0 THEN 1
END AS 'VAS',
CASE
WHEN phF LIKE 'never' THEN 0
WHEN phF LIKE 'sometimes' OR phF LIKE 'regularly' OR phF LIKE 'often' OR phF LIKE 'always' THEN 1
END AS 'phF',
CASE
WHEN MF LIKE 'never' THEN 0
WHEN MF LIKE 'sometimes' OR MF LIKE 'regularly' OR MF LIKE 'often' OR MF LIKE 'always' THEN 1
END AS 'MF',
CASE
WHEN ReIP LIKE 'worse' THEN -1
WHEN ReIP LIKE 'same' THEN 0
WHEN ReIP LIKE 'better' THEN 1
END AS 'ReIP',
CASE
WHEN sport LIKE 'No' THEN 0
WHEN sport LIKE 'Yes' THEN 1
END AS 'sport'
FROM questionnaires
'''
temp = sqldf(query)

# aggregate questionnaires into daily fatigue values
query = '''
SELECT subjectID, SUBSTRING(Timestamp, 1, 8) AS date, GROUP_CONCAT(DISTINCT timezone) AS timezone, GROUP_CONCAT(VAS) AS VAS, GROUP_CONCAT(ReIP) AS ReIP, GROUP_CONCAT(phF) AS phF, GROUP_CONCAT(MF) AS MF, GROUP_CONCAT(sport) AS sport, COUNT(*) AS n_answers
FROM temp
GROUP BY subjectID, date
HAVING n_answers > 1
ORDER BY n_answers DESC;
'''
print('days with multiple questionnaires:')
sqldf(query).fillna(value=pd.NA)

days with multiple questionnaires:


,subjectID,date,timezone,VAS,ReIP,phF,MF,sport,n_answers
0,24,06.02.18,CET,"1,1,1,0","0,0,0,-1","1,1,1,1","1,1,1,1",<NA>,4
1,24,07.02.18,CET,"0,0,1","0,0,-1","1,0,1","1,0,1",<NA>,3
2,4,30.03.19,UTC,"0,0","1,1","0,0","0,0",0.0,2
3,5,04.04.19,UTC,"1,0","1,1","1,0","1,0","0.0,0.0",2
4,5,07.04.19,UTC,"1,0","1,1","1,1","1,0","1.0,1.0",2
5,6,05.04.19,UTC,"0,0","1,0","1,1","1,1","0.0,0.0",2
6,8,09.06.19,UTC,"0,0","1,1","1,0","0,0","0.0,0.0",2
7,10,06.05.19,UTC,"0,0","1,0","1,1","0,0","0.0,0.0",2
8,10,08.05.19,UTC,"0,0","1,1","1,0","0,0","0.0,0.0",2
9,10,10.05.19,UTC,"0,0","-1,1","1,0","1,0","0.0,0.0",2


In [64]:
# aggregate questionnaires into SINGLE daily fatigue values
# TODO: CHECK VALIDNESS OF AVERAGING
query = '''
SELECT subjectID, SUBSTRING(Timestamp, 1, 8) AS date, GROUP_CONCAT(DISTINCT timezone) AS timezone, ROUND(AVG(VAS)) AS VAS, ROUND(AVG(ReIP)) AS ReIP, ROUND(AVG(phF)) AS phF, ROUND(AVG(MF)) AS MF, ROUND(AVG(sport)) AS sport, COUNT(*) AS n_answers
FROM temp
GROUP BY subjectID, date
ORDER BY n_answers DESC;
'''
Y = sqldf(query).fillna(value=pd.NA)

# for visualization:
query = '''
SELECT *
FROM Y
WHERE n_answers > 1
ORDER BY n_answers DESC;
'''
print('days with multiple questionnaires - averaged to single values:')
sqldf(query).fillna(value=pd.NA)

days with multiple questionnaires - averaged to single values:


,subjectID,date,timezone,VAS,ReIP,phF,MF,sport,n_answers
0,24,06.02.18,CET,1.0,0.0,1.0,1.0,<NA>,4
1,24,07.02.18,CET,0.0,0.0,1.0,1.0,<NA>,3
2,4,30.03.19,UTC,0.0,1.0,0.0,0.0,0.0,2
3,5,04.04.19,UTC,1.0,1.0,1.0,1.0,0.0,2
4,5,07.04.19,UTC,1.0,1.0,1.0,1.0,1.0,2
5,6,05.04.19,UTC,0.0,1.0,1.0,1.0,0.0,2
6,8,09.06.19,UTC,0.0,1.0,1.0,0.0,0.0,2
7,10,06.05.19,UTC,0.0,1.0,1.0,0.0,0.0,2
8,10,08.05.19,UTC,0.0,1.0,1.0,0.0,0.0,2
9,10,10.05.19,UTC,0.0,0.0,1.0,1.0,0.0,2


In [65]:
print('total number of days with fatigue data:', Y.shape[0])

total number of days with fatigue data: 476


# b) Physiological data

In [66]:
physio = physio.fillna(value=pd.NA) # otherwise SQL will ignore None values

# nested dataframe (all physiological data of the same day as lists)
query = '''
SELECT SubjectID AS subjectID,
    SUBSTRING(Timestamp, 1, 8) AS date,
    GROUP_CONCAT(ActivityCounts) AS ActivityCounts,
    GROUP_CONCAT(Barometer) AS Barometer,
    GROUP_CONCAT(BloodPerfusion) AS BloodPerfusion,
    GROUP_CONCAT(BloodPulseWave) AS BloodPulseWave,
    GROUP_CONCAT(EnergyExpenditure) AS EnergyExpenditure,
    GROUP_CONCAT(GalvanicSkinResponse) AS GalvanicSkinResponse,
    GROUP_CONCAT(HR) AS HR, GROUP_CONCAT(HRV) AS HRV,
    GROUP_CONCAT(RESP) AS RESP,
    GROUP_CONCAT(Steps) AS Steps,
    GROUP_CONCAT(SkinTemperature) AS SkinTemperature,
    GROUP_CONCAT(ActivityClass) AS ActivityClass
FROM physio
GROUP BY subjectID, date;'''
X = sqldf(query)
print('total number of days:', X.shape[0])

total number of days: 951


In [67]:
# days without sensory data
query = '''
SELECT *
FROM X
WHERE ActivityCounts IS NULL AND Barometer IS NULL AND BloodPerfusion IS NULL AND BloodPulseWave IS NULL AND EnergyExpenditure IS NULL AND GalvanicSkinResponse IS NULL AND HR IS NULL AND HRV IS NULL AND RESP IS NULL AND STEPS IS NULL AND SkinTemperature IS NULL AND ActivityClass IS NULL
'''
temp = sqldf(query)
print('number of discarded days:', temp.shape[0])

# discard days without sensory data
query = '''
SELECT *
FROM X

EXCEPT

SELECT *
FROM temp;
'''
X = sqldf(query)

number of discarded days: 406


In [68]:
print('total number of days with physiological data:', X.shape[0])

total number of days with physiological data 545


# Join data

# a) unnested data (one row <=> one physiological measurement)

In [69]:
query = '''
SELECT SubjectID AS subjectID,
    SUBSTRING(Timestamp, 1, 8) AS date,
    ActivityCounts,
    Barometer,
    BloodPerfusion,
    BloodPulseWave,
    EnergyExpenditure,
    GalvanicSkinResponse,
    HR,
    HRV,
    RESP,
    Steps,
    SkinTemperature,
    ActivityClass
FROM physio;
'''
temp = sqldf(query).fillna(value=pd.NA)

query = '''
SELECT Y.subjectID,
    Y.date,
    temp.ActivityCounts,
    temp.Barometer,
    temp.BloodPerfusion,
    temp.BloodPulseWave,
    temp.EnergyExpenditure,
    temp.GalvanicSkinResponse,
    temp.HR,
    temp.HRV,
    temp.RESP,
    temp.Steps,
    temp.SkinTemperature,
    temp.ActivityClass,
    y.timezone,
    y.VAS,
    y.phF,
    y.MF,
    y.ReIP,
    y.sport,
    y.n_answers
FROM temp JOIN Y ON temp.subjectID = Y.subjectID AND temp.date = Y.date;'''
data_unnested = sqldf(query)
print('total number of physiological measurements with corresp. fatigue value:', data_unnested.shape[0])

total number of physiological measurement with corresp. fatigue value: 617277


# b) nested data (one row <=> one day of physiological measurements)

In [70]:
query = '''
SELECT Y.subjectID, Y.date, X.ActivityCounts, X.Barometer, X.BloodPerfusion, X.BloodPulseWave, X.EnergyExpenditure, X.GalvanicSkinResponse, X.HR, X.HRV, X.RESP, X.Steps, X.SkinTemperature, X.ActivityClass, y.timezone, y.VAS, y.phF, y.MF, y.ReIP, y.sport, y.n_answers
FROM X JOIN Y ON X.subjectID = Y.subjectID AND X.date = Y.date;'''
data = sqldf(query)

print('number of discarded days with questionnaires but no physiological measurements:', Y.shape[0] - data.shape[0])

number of discarded days with questionnaires but no physiological measurements: 66


In [71]:
print('total number of days with physiological measurements + corresp. fatigue values:', data.shape[0])

total number of days with physiological measurements + corresp. fatigue values: 410


# Export data

In [72]:
output_path = '/'.join(path.split('/')[:-1]) + '/Output'

In [73]:
data.to_csv(output_path + '/combined_data.csv')

In [74]:
data_unnested.to_csv(output_path + '/combined_data_unnested.csv')